In [84]:
# Import necessary libraries
import numpy as np
import random
import time
from typing import List, Tuple, Optional
from copy import deepcopy
from math import sqrt, log

In [85]:
# Tic-Tac-Toe game environment
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0: empty, 1: player 1, -1: player 2

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)

    def is_valid_move(self, x: int, y: int) -> bool:
        return self.board[x, y] == 0

    def make_move(self, x: int, y: int, player: int) -> bool:
        if self.is_valid_move(x, y):
            self.board[x, y] = player
            return True
        return False

    def check_winner(self) -> Optional[int]:
        # Check rows, columns, and diagonals
        for i in range(3):
            if abs(sum(self.board[i, :])) == 3:
                return np.sign(sum(self.board[i, :]))
            if abs(sum(self.board[:, i])) == 3:
                return np.sign(sum(self.board[:, i]))
        if abs(self.board.trace()) == 3:
            return np.sign(self.board.trace())
        if abs(np.fliplr(self.board).trace()) == 3:
            return np.sign(np.fliplr(self.board).trace())
        return None if 0 in self.board else 0  # 0: draw, None: ongoing game

    def get_available_moves(self) -> List[Tuple[int, int]]:
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def display(self):
        display_board = self.board.astype(str)
        display_board[display_board == "0"] = "."
        display_board[display_board == "1"] = "X"
        display_board[display_board == "-1"] = "O"
        print("\n".join([" ".join(row) for row in display_board]))
        print()


In [86]:
# Minimax method
def minimax(state: TicTacToe, player: int) -> Tuple[int, int]:
    """
    Implement the Minimax algorithm with improved move selection
    """
    def minimax_score(is_maximizing):
        # Check Terminal State
        result = state.check_winner()
        if result is not None:
            if result == player:
                return 1000
            elif result == 0:
                return 0
            else:
                return -1000

        if is_maximizing:    # Check for Maximum among Successors
            best_score = float('-inf')
            available_successors = state.get_available_moves()
            
            for row, col in available_successors:
                        
                state.board[row, col] = player
                score = minimax_score(False)
                state.board[row, col] = 0
                
                best_score = max(score, best_score)

            return best_score
        
        else:    # Check for Min
            best_score = float('inf')
            opponent = -1 if player == 1 else 1
            available_successors = state.get_available_moves()
            for row, col in available_successors:
                
                state.board[row, col] = opponent
                score = minimax_score(True)
                state.board[row, col] = 0
                
                best_score = min(score, best_score)

            return best_score

    best_move = None
    best_score = float('-inf')
    available_moves = state.get_available_moves()
    for row, col in available_moves:
        
        state.board[row, col] = player
        score = minimax_score(False)
        state.board[row, col] = 0
        
        if score > best_score:
            best_score = score
            best_move = (row, col)
    
    return best_move

In [87]:
# Alpha-beta method
def alpha_beta(state: TicTacToe, player: int) -> Tuple[int, int]:
    """
    Implement the Alpha-Beta Pruning algorithm here.
    """
    def alpha_beta_score(alpha, beta, is_maximizing):
        # Check Terminal State
        result = state.check_winner()
        if result is not None:
            if result == player:
                return 1000
            elif result == 0:
                return 0
            else:
                return -1000

        if is_maximizing:    # Check for Maximum among Successors
            max_eval = float('-inf')
            
            available_successors = state.get_available_moves()
            for row, col in available_successors:
                        
                state.board[row, col] = player
                eval = alpha_beta_score(alpha, beta, False)
                state.board[row, col] = 0
                
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break

            return max_eval
        
        else:    # Check for Min
            min_eval = float('inf')
            opponent = -1 if player == 1 else 1
            available_successors = state.get_available_moves()
            for row, col in available_successors:
                
                state.board[row, col] = opponent
                eval = alpha_beta_score(alpha, beta,True)
                state.board[row, col] = 0
                
                min_eval = min(eval, min_eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break                

            return min_eval

    best_move = None
    best_score = float('-inf')
    alpha = float('-inf')
    beta = float('inf')
    available_moves = state.get_available_moves()
    for row, col in available_moves:
        
        state.board[row, col] = player
        score = alpha_beta_score(alpha, beta, False)
        state.board[row, col] = 0
        
        if score > best_score:
            best_score = score
            best_move = (row, col)
    
    return best_move

In [88]:
# evaluatiob based method
def evaluation_based(state: TicTacToe, player: int) -> Tuple[int, int]:
    """
    Implement a heuristic evaluation-based decision-making algorithm here.
    """
    def evaluate():
        lines = []
        line = []
        for i in range(3):
            line = [state.board[i, j] for j in range(3)]
            lines.append(line)
        for j in range(3):
            line = [state.board[i, j] for i in range(3)]
            lines.append(line)

        line = [state.board[i, i] for i in range(3)]
        lines.append(line)
        line = [state.board[i, 2 - i] for i in range(3)]
        lines.append(line)

        score = 0
        for line in lines:
            if line.count(player) == 3:
                return 1000000
            if line.count(player) == 2 and line.count(0) == 1:
                score += 200
            if line.count(player) == 2 and line.count(0) == 0:
                score -= 100
            if line.count(player) == 1 and line.count(-1 * player) < 2:
                score += 100
            if line.count(-1 * player) == 2 and line.count(0) == 1:
                score -= 1000000

        return score
            

    best_move = None
    best_score = float('-inf')
    available_moves = state.get_available_moves()
    for row, col in available_moves:
        state.board[row, col] = player
        score = evaluate()
        state.board[row, col] = 0

        if score > best_score:
            best_score = score
            best_move = row, col

    return best_move

In [89]:
# Monte Carlo Tree search method
def monte_carlo_tree_search(state: TicTacToe, player: int) -> Tuple[int, int]:
    """
    Implement the Monte Carlo Tree Search algorithm here.
    """
    class Node:
        def __init__(self, state: TicTacToe, parent=None, move=None):
            self.state = state
            self.t = 0
            self.n = 0
            self.move = move
            self.children = []
            self.parent = parent
            
    def rollout(game_node: Node, turn) -> int:
        match_res = game_node.state.check_winner()
        if match_res == player:
            return 1000
        elif match_res == (-1) * player:
            return -1000
        elif match_res == 0:
            return 0
        
        avail_moves = game_node.state.get_available_moves()
        if not avail_moves:
            return 0
            
        rnd_x, rnd_y = random.choice(avail_moves)
        game_copy = deepcopy(game_node)
        game_copy.state.board[rnd_x, rnd_y] = turn
        return rollout(game_copy, (-1) * turn)
    
    def backpropagate(value, node: Node):
        while node is not None:
            node.n += 1
            node.t += value
            node = node.parent
            
    def uct_score(node: Node, parent_n: int):
        if node.n == 0:
            return float('inf')
        return (node.t / node.n) + 2 * sqrt(log(parent_n) / node.n)

    root = Node(state)
    
    for _ in range(100):
        current = root
        while current.children and all(child.n > 0 for child in current.children):
            current = max(current.children, 
                         key=lambda x: uct_score(x, current.n))
            
        if current.n > 0:
            avail_moves = current.state.get_available_moves()
            if avail_moves and not current.children:
                for row, col in avail_moves:
                    tmp = deepcopy(current.state)
                    tmp.board[row, col] = player
                    current.children.append(Node(tmp, parent=current, move=(row, col)))
                current = random.choice(current.children)
                
        result = rollout(current, player)
        
        backpropagate(result, current)
    
    if not root.children:
        return random.choice(state.get_available_moves())
        
    best_node:Node = max(root.children, key=lambda x: x.n)
    return best_node.move

In [90]:
# Simulate a match between two methods
def simulate_game(method1, method2):
    game = TicTacToe()
    players = [1, -1]
    methods = {1: method1, -1: method2}
    while game.check_winner() is None:
        current_player = players[0]
        move = methods[current_player](game, current_player)
        game.make_move(*move, current_player)
        players.reverse()
    game.display()
    return game.check_winner()

In [92]:
# Main function for testing
if __name__ == "__main__":
# for _ in range(100):
    # Example match (methods need implementation)
    # winner = simulate_game(minimax, alpha_beta)
    # winner = simulate_game(minimax, minimax)
    # winner = simulate_game(minimax, evaluation_based)
    winner = simulate_game(monte_carlo_tree_search, evaluation_based)
    if winner == 1:
        print("Player 1 (Method 1) wins!")
    elif winner == -1:
        print("Player 2 (Method 2) wins!")
    else:
        print("It's a draw!")

Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 1 (Method 1) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
It's a draw!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
It's a draw!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Method 2) wins!
Player 2 (Me

In [93]:
def get_game_statistics(algorithm1, algorithm2, num_matches):
    """
    Run multiple matches between two algorithms and collect performance statistics.
    
    Args:
        algorithm1: First algorithm function (minimax, alpha_beta, etc.)
        algorithm2: Second algorithm function
        num_matches: Number of matches to play
    
    Returns:
        tuple: (alg1_name, alg2_name, alg1_wins, alg2_wins, ties, avg_time_alg1, avg_time_alg2)
    """
    alg1_name = algorithm1.__name__
    alg2_name = algorithm2.__name__
    alg1_wins = 0
    alg2_wins = 0
    ties = 0
    
    # Lists to store time measurements
    alg1_times = []
    alg2_times = []
    
    for match in range(num_matches):
        game = TicTacToe()
        players = [1, -1]
        methods = {1: algorithm1, -1: algorithm2}
        
        while game.check_winner() is None:
            current_player = players[0]
            current_method = methods[current_player]
            
            # Measure time for move calculation
            start_time = time.time()
            move = current_method(game, current_player)
            end_time = time.time()
            
            # Store time measurement for appropriate algorithm
            if current_player == 1:
                alg1_times.append(end_time - start_time)
            else:
                alg2_times.append(end_time - start_time)
            
            game.make_move(*move, current_player)
            players.reverse()
        
        # Record match result
        result = game.check_winner()
        if result == 1:
            alg1_wins += 1
        elif result == -1:
            alg2_wins += 1
        else:
            ties += 1
    
    # Calculate average move times
    avg_time_alg1 = sum(alg1_times) / len(alg1_times) if alg1_times else 0
    avg_time_alg2 = sum(alg2_times) / len(alg2_times) if alg2_times else 0
    
    return (
        alg1_name,
        alg2_name,
        alg1_wins,
        alg2_wins,
        ties,
        avg_time_alg1,
        avg_time_alg2
    )

In [ ]:
results = get_game_statistics(minimax, alpha_beta, 1)
print(f"Algorithm 1: {results[0]}")
print(f"Algorithm 2: {results[1]}")
print(f"Algorithm 1 wins: {results[2]}")
print(f"Algorithm 2 wins: {results[3]}")
print(f"Ties: {results[4]}")
print(f"Average move time Algorithm 1: {results[5]:.4f} seconds")
print(f"Average move time Algorithm 2: {results[6]:.4f} seconds")

Algorithm 1: minimax
Algorithm 2: alpha_beta
Algorithm 1 wins: 0
Algorithm 2 wins: 0
Ties: 1
Average move time Algorithm 1: 1.1752 seconds
Average move time Algorithm 2: 0.0116 seconds


In [96]:
results = get_game_statistics(evaluation_based, minimax, 1)
print(f"Algorithm 1: {results[0]}")
print(f"Algorithm 2: {results[1]}")
print(f"Algorithm 1 wins: {results[2]}")
print(f"Algorithm 2 wins: {results[3]}")
print(f"Ties: {results[4]}")
print(f"Average move time Algorithm 1: {results[5]:.4f} seconds")
print(f"Average move time Algorithm 2: {results[6]:.4f} seconds")

Algorithm 1: evaluation_based
Algorithm 2: minimax
Algorithm 1 wins: 0
Algorithm 2 wins: 0
Ties: 1
Average move time Algorithm 1: 0.0000 seconds
Average move time Algorithm 2: 0.1506 seconds


In [97]:
results = get_game_statistics(minimax, monte_carlo_tree_search, 30)
print(f"Algorithm 1: {results[0]}")
print(f"Algorithm 2: {results[1]}")
print(f"Algorithm 1 wins: {results[2]}")
print(f"Algorithm 2 wins: {results[3]}")
print(f"Ties: {results[4]}")
print(f"Average move time Algorithm 1: {results[5]:.4f} seconds")
print(f"Average move time Algorithm 2: {results[6]:.4f} seconds")

Algorithm 1: minimax
Algorithm 2: monte_carlo_tree_search
Algorithm 1 wins: 30
Algorithm 2 wins: 0
Ties: 0
Average move time Algorithm 1: 1.5446 seconds
Average move time Algorithm 2: 0.0495 seconds


In [98]:
results = get_game_statistics(alpha_beta, evaluation_based, 1)
print(f"Algorithm 1: {results[0]}")
print(f"Algorithm 2: {results[1]}")
print(f"Algorithm 1 wins: {results[2]}")
print(f"Algorithm 2 wins: {results[3]}")
print(f"Ties: {results[4]}")
print(f"Average move time Algorithm 1: {results[5]:.4f} seconds")
print(f"Average move time Algorithm 2: {results[6]:.4f} seconds")

Algorithm 1: alpha_beta
Algorithm 2: evaluation_based
Algorithm 1 wins: 0
Algorithm 2 wins: 0
Ties: 1
Average move time Algorithm 1: 0.0798 seconds
Average move time Algorithm 2: 0.0000 seconds


In [100]:
results = get_game_statistics(monte_carlo_tree_search, alpha_beta, 30)
print(f"Algorithm 1: {results[0]}")
print(f"Algorithm 2: {results[1]}")
print(f"Algorithm 1 wins: {results[2]}")
print(f"Algorithm 2 wins: {results[3]}")
print(f"Ties: {results[4]}")
print(f"Average move time Algorithm 1: {results[5]:.4f} seconds")
print(f"Average move time Algorithm 2: {results[6]:.4f} seconds")

Algorithm 1: monte_carlo_tree_search
Algorithm 2: alpha_beta
Algorithm 1 wins: 0
Algorithm 2 wins: 24
Ties: 6
Average move time Algorithm 1: 0.0453 seconds
Average move time Algorithm 2: 0.0182 seconds


In [114]:
results = get_game_statistics(monte_carlo_tree_search, evaluation_based, 100)
print(f"Algorithm 1: {results[0]}")
print(f"Algorithm 2: {results[1]}")
print(f"Algorithm 1 wins: {results[2]}")
print(f"Algorithm 2 wins: {results[3]}")
print(f"Ties: {results[4]}")
print(f"Average move time Algorithm 1: {results[5]:.4f} seconds")
print(f"Average move time Algorithm 2: {results[6]:.4f} seconds")

Algorithm 1: monte_carlo_tree_search
Algorithm 2: evaluation_based
Algorithm 1 wins: 4
Algorithm 2 wins: 86
Ties: 10
Average move time Algorithm 1: 0.0510 seconds
Average move time Algorithm 2: 0.0000 seconds
